In [ ]:
import pandas as pd
df = pd.read_csv('../data/china-social-news.csv')

In [2]:
df.set_index(pd.to_datetime(df['时间']), inplace=True)
df.dropna(inplace=True)
df.sort_index(inplace=True)

In [3]:
print('数据集大小：', df.shape[0])
print('时间跨度:', df.index.min().date(), df.index.max().date())

数据集大小： 1334
时间跨度: 2020-01-26 2020-05-19


In [4]:
df.head(3)

,标题,时间,URL,正文内容,来源
时间,,,,,
2020-01-26,中国生物多样性保护与绿色发展基金会：向湖北紧急支援4万个口罩,2020-01-26,http://www.chinanpo.gov.cn/1944/123496/nextind...,大年三十，中国绿发会紧急采购4万个口罩，支援武汉市和襄阳市，今天首批12000个口罩已到达湖...,中国生物多样性保护与绿色发展基金会
2020-01-26,中国旅行社协会,2020-01-26,http://www.chinanpo.gov.cn/1944/123533/nextind...,：处理行前解约应注意这几点针对新型冠状病毒肺炎疫情，积极响应国家相关部门的通知要求，发挥法律...,中国旅行社协会
2020-01-26,中国旅行社协会,2020-01-26,http://www.chinanpo.gov.cn/1944/123530/nextind...,：致境内外旅游供应商、旅游业者的一封公开信,中国旅行社协会


In [5]:
from jieba import analyse
def get_keywords(news_list):
    keywords = []
    for keyword in analyse.extract_tags(''.join(news_list), topK=100, withWeight=True):
        keywords.append(keyword)
    return keywords

In [6]:
from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

def render_wordcloud(data, title):
    c = (
        WordCloud()
        .add(series_name="", data_pair=data, word_size_range=[20, 100], shape=SymbolType.ROUND_RECT)
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title=title, title_textstyle_opts=opts.TextStyleOpts(font_size=23)
            ),
            tooltip_opts=opts.TooltipOpts(is_show=True),
        )
    )
    return c

In [7]:
from pyecharts.charts import Timeline

t = (
    Timeline(init_opts=opts.InitOpts(width='1100px', height='800px'))
    .add_schema(play_interval=500)
)

In [8]:
def add_to_timeline(data, date):
    keywords = get_keywords(data)
    c = render_wordcloud(keywords, '疫情词云图')
    t.add(c, date.date())

In [9]:
def get_timeline():
    result = []
    count = 0
    MAX = 80
    date = pd.to_datetime('2020-01-26')
    for date in pd.date_range(start='2020-01-26', end='2020-05-19'):
        try:
            count = count + df.loc[date, '正文内容'].shape[0]
            for item in df.loc[date, '正文内容'].tolist():
                result.append(item)
            if count >= MAX:
                add_to_timeline(result, date)
                result.clear()
                count = 0
        except Exception:
            continue
    if count > 0:
        add_to_timeline(result, date)

In [10]:
get_timeline()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\49518\AppData\Local\Temp\jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built successfully.


# 中国社会组织公共服务平台疫情防控专区新闻词云可视化

我们通过爬虫技术，对中国社会组织公共服务平台疫情防控专区新闻进行了抓取，共计获得了 1-26 至 5-19 时间段内共1400+新闻；

词云图，也叫文字云，是对文本中出现频率较高的“关键词”予以视觉化的展现，词云图过滤掉大量的低频低质的文本信息，使得浏览者只要一眼扫过文本就可领略文本的主旨。

对该平台上发布的所有文章使用jieba进行分词、获取主题词（取排名前100位），并渲染词云图：

In [11]:
t.load_javascript()

In [12]:
t.render_notebook()

- 疫情前期主题词以“防控”、“组织”、“工作”、“社会”为主，对应于主要的抗疫力量为政府动员工作和社会组织捐赠
- 从2月末开始，“企业”，“复工”等词语占比越来越大，也对应着复工成为主要需求；

## TF-IDF值
TF-IDF（Term Frequency-InversDocument Frequency）是一种常用于信息处理和数据挖掘的加权技术。该技术采用一种统计方法，根据字词的在文本中出现的次数和在整个语料中出现的文档频率来计算一个字词在整个语料中的重要程度。它的优点是能过滤掉一些常见的却无关紧要本的词语，同时保留影响整个文本的重要字词。

TF - IDF = TF * IDF

TF（Term Frequency）表示某个关键词在整篇文章中出现的频率。IDF（InversDocument Frequency）表示计算倒文本频率。文本频率是指某个关键词在整个语料所有文章中出现的次数。倒文档频率又称为逆文档频率，它是文档频率的倒数，主要用于降低所有文档中一些常见却对文档影响不大的词语的作用。

In [13]:
from pyecharts.charts import Bar

def get_tf_bar(x_data, y_data):
    b = (
        Bar(
            init_opts=opts.InitOpts()
        )
        .add_xaxis(x_data)
        .add_yaxis(
            series_name='',
            y_axis=y_data
        )
        .add_dataset()
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title='TF-IDF Ranking', pos_left='35%'),
            legend_opts=opts.LegendOpts(pos_right='10%', pos_bottom='10%'),
            tooltip_opts=opts.TooltipOpts(trigger='axis')
        )
    )
    return b

In [14]:
keywords = get_keywords(df.loc[:, '正文内容'])
keywords[:10]

[('疫情', 0.22706395346709843),
 ('防控', 0.17074977030094585),
 ('组织', 0.0675057784424657),
 ('捐赠', 0.062243513190542804),
 ('社会', 0.057935903583603246),
 ('工作', 0.057022622823337914),
 ('协会', 0.054245645974525636),
 ('防疫', 0.04727786567173266),
 ('冠状病毒', 0.04383926161901604),
 ('肺炎', 0.04318461971307456)]

In [15]:
x_data = []
y_data = []

for keyword in keywords[:20]:
    x_data.append(keyword[0])
    y_data.append(keyword[1])
    
x_data.reverse()
y_data.reverse()

In [16]:
b = get_tf_bar(x_data, y_data)

In [17]:
b.load_javascript()

In [18]:
b.render_notebook()

输出结果如下图所示，可以看到“疫情”、“组织”、“捐赠”、“社会”、“协会”、“肺炎”、“物资”等都是高频词，也是大众普 遍关心的主题。